In [1]:
import yfinance as yf
import pandas as pd

import config as cfg

In [2]:
# Download stock prices
stock_data = yf.download(cfg.tickers, start=cfg.start_date, end=cfg.end_date)

# Select 'Close' prices for each stock
close_prices = stock_data['Close']

# Transform stock prices to a dataframe
df = pd.DataFrame(close_prices)
print(df.head(10))

[*********************100%***********************]  4 of 4 completed
                AB.PA      CA.PA      CO.PA        ^FCHI
Date                                                    
2021-01-04  18.799999  14.490000  25.639999  5588.959961
2021-01-05  18.719999  15.010000  25.920000  5564.600098
2021-01-06  17.900000  15.340000  27.889999  5630.600098
2021-01-07  17.760000  15.420000  27.830000  5669.850098
2021-01-08  17.680000  15.240000  27.580000  5706.879883
2021-01-11  17.299999  15.325000  27.400000  5662.430176
2021-01-12  17.040001  15.465000  27.490000  5650.970215
2021-01-13  16.420000  17.540001  28.559999  5662.669922
2021-01-14  16.600000  17.100000  28.030001  5681.140137
2021-01-15  16.540001  16.610001  27.840000  5611.689941


In [3]:
results = []

# Repeat for each period of watch_days
for watch_days in range(1, 500, 2):  # watch_days represents the number of days for price increase calculation
    # num_iterations = (len(df) - watch_days + 1) // watch_days # Calculate the number of iterations
    for hold_days in range(1, 500, 2):  # hold_days represents the number of additional days before selling. Repeat for each period of watch_days plus hold_days
        num_iterations = int((len(df) - hold_days) / watch_days) - 1

        for num_stocks_to_buy in range(1, 40):  # num_stocks_to_buy represents the number of stocks to buy
            for max_loss in range(0, 200, 5):  # max_loss represents the percentage decrease threshold for selling
                total_profit = 1

                for i in range(num_iterations):
                    start_watch_day_number = i * watch_days
                    buy_day_number = start_watch_day_number + watch_days
                    sell_day_number = buy_day_number + hold_days
                    # print("start_watch_day_number:", start_watch_day_number, ", buy_day_number:", buy_day_number, ", sell_day_number:", sell_day_number)

                    price_increase = df[start_watch_day_number:buy_day_number].pct_change(watch_days - 1).tail(1) # Calculate the price increase in the last watch_days days for each stock
                    # print("price_increase: ________________________________ \n", price_increase)

                    top_stocks = price_increase.squeeze().nlargest(num_stocks_to_buy).index # Select the num_stocks_to_buy stocks with the highest price increase
                    # print("top_stocks: ____________________________________ \n", top_stocks)

                    buy_prices = df.loc[df.index[buy_day_number], top_stocks] # Calculate the buying prices at the start of the hold period
                    # print("buy_prices: ____________________________________ \n", buy_prices)
                    
                    sell_prices = df.loc[df.index[sell_day_number], top_stocks] # Calculate the selling prices after watch_days + hold_days days
                    # print("sell_prices: ___________________________________ \n", sell_prices)

                    profits = (sell_prices / buy_prices - 1) * 100 # Calculate the profit for each stock

                    sell_mask = (sell_prices / buy_prices - 1) < -max_loss / 100 # Filter the stocks to sell based on decrease threshold max_loss
                    profits[sell_mask] = -max_loss  # Set profit to -max_loss for stocks that decrease beyond threshold

                    total_profit *= 1 + profits.mean() / 100 # Calculate the total profit for the selected stocks (i.e. average of profits)
                    print('log iteration:', i, ', mean_profit:', profits.mean(), ', total_profit:', total_profit, ', watch_days:', watch_days, ', num_stocks_to_buy:', num_stocks_to_buy, ', hold_days:', hold_days,
                    ', max_loss:', max_loss,', top_stocks:', top_stocks)

                results.append({'watch_days': watch_days, 'num_stocks_to_buy': num_stocks_to_buy, 'hold_days': hold_days,
                                'max_loss': max_loss, 'total_profit': total_profit})

results_df = pd.DataFrame(results)

log iteration: 0 , mean_profit: 0.0 , total_profit: 1.0 , watch_days: 3 , num_stocks_to_buy: 1 , hold_days: 1 , max_loss: 0 , top_stocks: Index(['CO.PA'], dtype='object')
log iteration: 1 , mean_profit: 0.20703890812443504 , total_profit: 1.0020703890812444 , watch_days: 3 , num_stocks_to_buy: 1 , hold_days: 1 , max_loss: 0 , top_stocks: Index(['^FCHI'], dtype='object')
log iteration: 2 , mean_profit: 0.0 , total_profit: 1.0020703890812444 , watch_days: 3 , num_stocks_to_buy: 1 , hold_days: 1 , max_loss: 0 , top_stocks: Index(['CA.PA'], dtype='object')
log iteration: 3 , mean_profit: 0.0 , total_profit: 1.0020703890812444 , watch_days: 3 , num_stocks_to_buy: 1 , hold_days: 1 , max_loss: 0 , top_stocks: Index(['AB.PA'], dtype='object')
log iteration: 4 , mean_profit: 1.3868025774084636 , total_profit: 1.0159671270644701 , watch_days: 3 , num_stocks_to_buy: 1 , hold_days: 1 , max_loss: 0 , top_stocks: Index(['CO.PA'], dtype='object')
log iteration: 5 , mean_profit: 0.2865326772472976 , t

In [5]:
results_df.sort_values("total_profit", ascending=False, inplace=True)
print(results_df.to_markdown())

|     |   watch_days |   num_stocks_to_buy |   hold_days |   max_loss |   total_profit |
|----:|-------------:|--------------------:|------------:|-----------:|---------------:|
|  27 |            3 |                   1 |           4 |          0 |      24.7931   |
|  18 |            3 |                   1 |           3 |          0 |      16.1745   |
|  30 |            3 |                   2 |           4 |          0 |      15.2689   |
|  33 |            3 |                   3 |           4 |          0 |      12.8842   |
|  63 |            4 |                   1 |           4 |          0 |      11.761    |
|  21 |            3 |                   2 |           3 |          0 |      10.1348   |
|  24 |            3 |                   3 |           3 |          0 |       8.21427  |
|  54 |            4 |                   1 |           3 |          0 |       8.13647  |
|  28 |            3 |                   1 |           4 |          2 |       7.47997  |
|   9 |            3 